<a href="https://colab.research.google.com/github/Absolutorcinus/Histogram_ct_scan/blob/master/notebooks/spacy_new_model_training_from_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall spacy
!pip install spacy
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf
!python -m spacy download de_core_news_lg


In [1]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.3                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_trf (3.2.0), de_core_news_lg (3.2.0), en_core_web_lg (3.2.0)



In [3]:

'''
train = "/content/train_data.spacy"
dev = "/content/valid_data.spacy"
'''

'\ntrain = "/content/train_data.spacy"\ndev = "/content/valid_data.spacy"\n'

In [2]:
import pandas as pd
import spacy
import en_core_web_lg
import en_core_web_trf
from spacy.tokens import DocBin


import warnings
from pathlib import Path




In [3]:
#For the other method:

#create_spacy dataset
#en_spacy_set = [(t,{"entities":[(0, len(t), str(s))]}) for t,s in list(zip(en_df.text,en_df.sentiment))]
#create_spacy dataset
#en_spacy_set_m = [(t,{"entities":[(0, len(t), str(s))]}) for t,s in list(zip(en_df.text,en_df.sentiment))]


In [3]:
#en_df['sentiment'] = en_df.sentiment.map({'neg':0, 'pos':1})
#de_df['sentiment'] = de_df.sentiment.map({'neg':0, 'pos':1})
en_df = pd.read_csv("/content/drive/MyDrive/data/en_df_final.csv")[['text','sentiment']]
de_df = pd.read_csv("/content/drive/MyDrive/data/de_df_final.csv")[['text','sentiment']]
en_df['sentiment'] = en_df.sentiment.map({0:'neg', 1:'pos'})
de_df['sentiment'] = de_df.sentiment.map({0:'neg', 1:'pos'})

In [4]:
#this function gets dataframe and return list of training validating and testing datasets

def create_dataset(en_df):
  #shuffeling
  en_df = en_df.sample(frac=1).reset_index(drop=True)

  #70% for training dataset
  en_data_train = en_df[:int(len(en_df)*70/100)]

  #15% for validation dataset
  min_index = int(len(en_df)*70/100)
  max_index = int(len(en_df)*70/100)+int(len(en_df)*15/100)
  en_data_valid = en_df[min_index:max_index]

  #15% for teting dataset
  en_data_test = en_df[max_index:]

  #creating  training validation and  testing 
  en_data_train = [(t,s) for t,s in list(zip(en_data_train.text, en_data_train.sentiment))]
  en_data_valid = [(t,s) for t,s in list(zip(en_data_valid.text, en_data_valid.sentiment))]
  en_data_test = [(t,s) for t,s in list(zip(en_data_test.text, en_data_test.sentiment))]
  print(f"sum of the length of the dataset: {len(en_data_train)+len(en_data_valid)+len(en_data_test)}")

  return [en_data_train,en_data_valid,en_data_test ]



In [5]:
"""
this function will take a list((text,labels ))
returns  as spacy documents

input: list(tuple(text, label))
return: List(<spacy.Doc.doc>)
"""
def make_docs(list_text_label, language):
  if language == "en":
    nlp = spacy.load("en_core_web_lg")
  elif language == "de":
    nlp = spacy.load("de_core_news_lg")

  docs = []
  for doc, label in nlp.pipe(list_text_label, as_tuples=True):
    if label == "neg":
      doc.cats["positive"] = 0
      doc.cats["nagative"] = 1
    else:
      doc.cats["positive"] = 1
      doc.cats["nagative"] = 0
  # its reading the data and 
    docs.append(doc)
  return  docs
    

In [6]:
#creating binary files: traiing and validation for the training purposes
def create_binary_spacy(data_train, data_valid, language ):
  train_docs = make_docs(data_train, language)
  doc_bin = DocBin(docs=train_docs)
  doc_bin.to_disk(language+"_train_data.spacy")

  valid_docs = make_docs(data_valid, language)
  doc_bin = DocBin(docs=valid_docs)
  doc_bin.to_disk(language+"_valid_data.spacy")

## English

In [39]:
#****done*****

## German

In [7]:
#creating  german datasets
ds = create_dataset(de_df)

sum of the length of the dataset: 38471


In [8]:
#generate the files
create_binary_spacy(ds[0], ds[1], "de" )

In [9]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
!python -m spacy train config.cfg --output ./output --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-03-28 23:50:02,694] [INFO] Set up nlp object from config
[2022-03-28 23:50:02,705] [INFO] Pipeline: ['textcat_multilabel']
[2022-03-28 23:50:02,709] [INFO] Created vocabulary
[2022-03-28 23:50:02,710] [INFO] Finished initializing nlp object
[2022-03-28 23:50:18,896] [INFO] Initialized pipeline components: ['textcat_multilabel']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.0
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.51       50.00    0.50
  0     200          47.29       68.25    0.68
  1     400          47.27       69.68    0.70
  2     600          45.04       69.38    0.69
  3     800          45.12       69.56    0.70
  4    1000          43.91       69.66   

('quiz facebook genie wusste', 'pos')

In [15]:
nlp =  spacy.load("/content/drive/MyDrive/models/german_model/model-best")


In [28]:
predicted = list()
for i in range(len(ds[2])):
  doc = nlp(ds[2][i][0])
  if doc.cats["positive"] > doc.cats["nagative"]:
    predicted.append('pos')#{'pos':doc.cats["positive"]})
  elif doc.cats["positive"] <= doc.cats["nagative"]:
    predicted.append('neg')#{'neg':doc.cats["nagative"]})



In [43]:
sentiments = ([j for i,j in ds[2]])
predicted = (predicted)

count_pos_neg = 0
count_pos_pos = 0
count_neg_neg = 0
count_neg_pos = 0
count=0
for i in range(len(sentiments)):
  if sentiments[i] == predicted[i]:
    count+=1


print(count, len(sentiments))
  


3997 5772
